In [38]:
import pdftotext
# import pdfplumber

In [16]:
%config Completer.use_jedi = False

In [7]:
file = './pdfs/Curato-UsersManual.pdf'

In [14]:
# with pdfplumber.open(file) as pdf:
#     first_page = pdf.pages[0]
#     print(first_page.chars[0])

{'fontname': 'MYJSDS+ArialMT', 'adv': Decimal('0.944'), 'upright': True, 'x0': Decimal('158.000'), 'y0': Decimal('603.488'), 'x1': Decimal('182.544'), 'y1': Decimal('629.488'), 'width': Decimal('24.544'), 'height': Decimal('26.000'), 'size': Decimal('26.000'), 'object_type': 'char', 'page_number': 1, 'stroking_color': None, 'non_stroking_color': [0.8980392, 0, 0], 'text': 'W', 'top': Decimal('162.512'), 'bottom': Decimal('188.512'), 'doctop': Decimal('162.512')}


In [40]:
%%timeit
with open(file, 'rb') as pdf:
    curr_pdf = pdftotext.PDF(pdf, "secret")

225 µs ± 2.92 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [28]:
list(curr_pdf)
single_string_pdf = '\n\n'.join(curr_pdf)
single_string_pdf

'Wound Treatment System\n    User’s Manual\n\n\nCurato™ System User’s Manual                                                                     i\nCAUTION: This Curato™ Wound Treatment System User’s Manual is not a guarantee or\nwarranty. It is intended only as an operational guide. For additional information and\nquestions, please contact Infiniti Medicals Customer Service department\nat 1-650-327-5000.\nIn order for the Curato™ Wound Treatment System to provide safe, reliable, and proper\nperformance, the following condItions must be met. Failure to comply with these\ncondItions will void all pertinent warranties.\n   There are no user serviceable components in the Curato™. All assembly, operation,\n   adjustment, modification, maintenance, and/or repair must be carried out only by\n   qualified personnel authorized by Infiniti Medical.\n   The electrical installation of the room in which the device will be used must comply with the\n   appropriate national electrical standards.\n  

In [31]:
import re
import string

In [51]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', ' ', text) #brackets
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text) #punctions
    text = re.sub('\w*\d\w*', ' ', text) #digits
    text = re.sub('[’’“”…]', ' ', text) #quotes
    text = re.sub('\n', ' ', text) #newlines
    text = re.sub('♪', ' ', text) #symbols
    text = re.sub('–', ' ', text) #dashes
    return text

In [52]:
clean_text(single_string_pdf)

'wound treatment system     user s manual   curato™ system user s manual                                                                     i caution  this curato™ wound treatment system user s manual is not a guarantee or warranty  it is intended only as an operational guide  for additional information and questions  please contact infiniti medicals customer service department at          in order for the curato™ wound treatment system to provide safe  reliable  and proper performance  the following conditions must be met  failure to comply with these conditions will void all pertinent warranties     there are no user serviceable components in the curato™  all assembly  operation     adjustment  modification  maintenance  and or repair must be carried out only by    qualified personnel authorized by infiniti medical     the electrical installation of the room in which the device will be used must comply with the    appropriate national electrical standards     the product must be use

In [53]:
for i in curr_pdf:
    print(i)

Wound Treatment System
    User’s Manual

Curato™ System User’s Manual                                                                     i
CAUTION: This Curato™ Wound Treatment System User’s Manual is not a guarantee or
warranty. It is intended only as an operational guide. For additional information and
questions, please contact Infiniti Medicals Customer Service department
at 1-650-327-5000.
In order for the Curato™ Wound Treatment System to provide safe, reliable, and proper
performance, the following condItions must be met. Failure to comply with these
condItions will void all pertinent warranties.
   There are no user serviceable components in the Curato™. All assembly, operation,
   adjustment, modification, maintenance, and/or repair must be carried out only by
   qualified personnel authorized by Infiniti Medical.
   The electrical installation of the room in which the device will be used must comply with the
   appropriate national electrical standards.
   The product must b

In [55]:
import pandas as pd

In [72]:
array = []

In [73]:
for i in curr_pdf:
    array.append(clean_text(i).split())

In [74]:
len(curr_pdf)

25

In [75]:
pd.DataFrame(array)

,0,1,2,3,4,5,6,7,8,9,...,396,397,398,399,400,401,402,403,404,405
0,wound,treatment,system,user,s,manual,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,curato™,system,user,s,manual,i,caution,this,curato™,wound,...,None,None,None,None,None,None,None,None,None,None
2,curato™,system,user,s,manual,i,table,of,contents,introduction,...,None,None,None,None,None,None,None,None,None,None
3,curato™,system,user,s,manual,introduction,indications,the,curato™,wound,...,None,None,None,None,None,None,None,None,None,None
4,curato™,system,user,s,manual,safety,tips,the,curato™,wound,...,None,None,None,None,None,None,None,None,None,None
5,curato™,system,user,s,manual,body,cavity,wounds,underlying,structures,...,None,None,None,None,None,None,None,None,None,None
6,curato™,system,user,s,manual,features,easy,to,use,one,...,None,None,None,None,None,None,None,None,None,None
7,curato™,system,user,s,manual,care,cleaning,introduction,the,following,...,None,None,None,None,None,None,None,None,None,None
8,curato™,system,user,s,manual,cleaning,the,curato™,device,perform,...,None,None,None,None,None,None,None,None,None,None
9,curato™,system,user,s,manual,patient,care,it,is,recommended,...,None,None,None,None,None,None,None,None,None,None


In [78]:
import numpy as np

In [87]:
[len(set(i)) for i in array]

[6,
 128,
 57,
 190,
 152,
 106,
 203,
 114,
 180,
 148,
 108,
 65,
 187,
 99,
 92,
 160,
 151,
 154,
 142,
 64,
 113,
 8,
 62,
 71,
 5]

### thought: what if we made this into a browser extension?

In [88]:
pip install pdftotree

     |████████████████████████████████| 56 kB 4.0 MB/s eta 0:00:01
  Using cached sklearn-0.0.tar.gz (1.1 kB)
     |████████████████████████████████| 11.7 MB 12.2 MB/s eta 0:00:01
     |████████████████████████████████| 24.9 MB 20.6 MB/s eta 0:00:01
     |████████████████████████████████| 30.2 MB 31.2 MB/s eta 0:00:01
  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)


     |████████████████████████████████| 303 kB 40.0 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 165 kB 37.4 MB/s eta 0:00:01
  Using cached imageio-2.9.0-py3-none-any.whl (3.3 MB)
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=c0b9bf261f5be13a1a91bc607194b9986f5087503a9c37074f9d059da5f2140e
  Stored in directory: /home/vince/.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
  Created wheel for selectivesearch: filename=selectivesearch-0.4-py3-none-any.whl size=4359 sha256=343d4614bdc89a2b0bc63f09ecf02f5255bdc16e897f1d0dce792670970b4004
  Stored in directory: /home/vince/.cache/pip/wheels/66/4e/88/6de23ce74be839a953498c4ebdfa809ad7da9422ac89ae856c
Successfully built sklearn selectivesearch
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pdftotree

In [3]:
import pdfplumber

In [4]:
cleaned_pdf.extract_text(x_tolerance=3, y_tolerance=3)

NameError: name 'cleaned_pdf' is not defined

In [45]:
import re
import string

In [21]:
pdfplumber.open(file)

In [36]:
%%timeit
pages = []
with pdfplumber.open(file) as pdf:
    for i in range(len(pdf.pages)):
        pages.append(pdf.pages[i].extract_text())

966 ms ± 9.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
len(pages)

25

TypeError: 'function' object is not subscriptable

In [35]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', ' ', text) #brackets
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text) #punctions
    text = re.sub('\w*\d\w*', ' ', text) #digits
    text = re.sub('[’’“”…]', ' ', text) #quotes
    text = re.sub('\n', ' ', text) #newlines
    text = re.sub('♪', ' ', text) #symbols
    text = re.sub('–', ' ', text) #dashes
    return text

In [77]:
cleaned_text = []
for i in curr_pdf:
    cleaned_text.append(clean_text(i))
[len(set(i)) for i in cleaned_text]

[14,
 27,
 26,
 29,
 26,
 29,
 28,
 27,
 27,
 27,
 26,
 24,
 28,
 26,
 27,
 27,
 27,
 27,
 27,
 26,
 29,
 19,
 24,
 32,
 14]

In [78]:
cleaned_text[0]

'wound treatment system     user s manual '

In [81]:
clean_tmp = []



[['wound', 'treatment', 'system', 'user', 's', 'manual'],
 ['curato™',
  'system',
  'user',
  's',
  'manual',
  'i',
  'caution',
  'this',
  'curato™',
  'wound',
  'treatment',
  'system',
  'user',
  's',
  'manual',
  'is',
  'not',
  'a',
  'guarantee',
  'or',
  'warranty',
  'it',
  'is',
  'intended',
  'only',
  'as',
  'an',
  'operational',
  'guide',
  'for',
  'additional',
  'information',
  'and',
  'questions',
  'please',
  'contact',
  'infiniti',
  'medicals',
  'customer',
  'service',
  'department',
  'at',
  'in',
  'order',
  'for',
  'the',
  'curato™',
  'wound',
  'treatment',
  'system',
  'to',
  'provide',
  'safe',
  'reliable',
  'and',
  'proper',
  'performance',
  'the',
  'following',
  'conditions',
  'must',
  'be',
  'met',
  'failure',
  'to',
  'comply',
  'with',
  'these',
  'conditions',
  'will',
  'void',
  'all',
  'pertinent',
  'warranties',
  'there',
  'are',
  'no',
  'user',
  'serviceable',
  'components',
  'in',
  'the',
  'cura

In [72]:
cleaned_pdf

[['wound', 'treatment', 'system', 'user', 's', 'manual'],
 ['curato™',
  'system',
  'user',
  's',
  'manual',
  'i',
  'caution',
  'this',
  'curato™',
  'wound',
  'treatment',
  'system',
  'user',
  's',
  'manual',
  'is',
  'not',
  'a',
  'guarantee',
  'or',
  'warranty',
  'it',
  'is',
  'intended',
  'only',
  'as',
  'an',
  'operational',
  'guide',
  'for',
  'additional',
  'information',
  'and',
  'questions',
  'please',
  'contact',
  'infiniti',
  'medicals',
  'customer',
  'service',
  'department',
  'at',
  'in',
  'order',
  'for',
  'the',
  'curato™',
  'wound',
  'treatment',
  'system',
  'to',
  'provide',
  'safe',
  'reliable',
  'and',
  'proper',
  'performance',
  'the',
  'following',
  'conditions',
  'must',
  'be',
  'met',
  'failure',
  'to',
  'comply',
  'with',
  'these',
  'conditions',
  'will',
  'void',
  'all',
  'pertinent',
  'warranties',
  'there',
  'are',
  'no',
  'user',
  'serviceable',
  'components',
  'in',
  'the',
  'cura

In [52]:
import pandas as pd

In [58]:
pdf_data_frame = pd.DataFrame(cleaned_pdf)

In [55]:
import matplotlib.pyplot as plt

In [62]:
import dataframe_image as dfi

In [64]:
pdf_data_frame.dfi.export('df.png', max_cols=-1)

In [69]:
len(set([item for i in cleaned_pdf for item in i]))

1084

In [70]:
cleaned_pdf[:3]

[['wound', 'treatment', 'system', 'user', 's', 'manual'],
 ['curato™',
  'system',
  'user',
  's',
  'manual',
  'i',
  'caution',
  'this',
  'curato™',
  'wound',
  'treatment',
  'system',
  'user',
  's',
  'manual',
  'is',
  'not',
  'a',
  'guarantee',
  'or',
  'warranty',
  'it',
  'is',
  'intended',
  'only',
  'as',
  'an',
  'operational',
  'guide',
  'for',
  'additional',
  'information',
  'and',
  'questions',
  'please',
  'contact',
  'infiniti',
  'medicals',
  'customer',
  'service',
  'department',
  'at',
  'in',
  'order',
  'for',
  'the',
  'curato™',
  'wound',
  'treatment',
  'system',
  'to',
  'provide',
  'safe',
  'reliable',
  'and',
  'proper',
  'performance',
  'the',
  'following',
  'conditions',
  'must',
  'be',
  'met',
  'failure',
  'to',
  'comply',
  'with',
  'these',
  'conditions',
  'will',
  'void',
  'all',
  'pertinent',
  'warranties',
  'there',
  'are',
  'no',
  'user',
  'serviceable',
  'components',
  'in',
  'the',
  'cura

In [ ]:
[item for i in cleaned_pdf for item in i]